In [69]:
import numpy as np
import random 
from scipy.stats import unitary_group

# defining Pauli matrices & CNOT
Y = np.array([[0,-1j],[1j,0]])
Z = np.array([[1,0],[0,-1]])
CNOT = [[1,0,0,0],[0,0,0,1],[0,0,1,0],[0,1,0,0]] # switch tensor control

# defining rotation gates
def Ry(A):
    M = np.array([[np.cos(A/2),-np.sin(A/2)],[np.sin(A/2), np.cos(A/2)]])
    return M
def Rz(A):
    M = np.array([[np.exp(-1j*A/2),0],[0,np.exp(1j*A/2)]])
    return M
    
# Parameterising the general unitary with rotation gates
def U (A0,A1,A2): # Arguments: Euler Angles
    M = Rz(A1)@Ry(A0)@Rz(A2)
    return M

# derivatives wrt A0,A1,A2
def DA0(A0,A1,A2):
    dA0 = (-1j/2)*(Rz(A1)@Y@Ry(A0)@Rz(A2))
    return dA0
def DA1(A0,A1,A2):
    dA1 = (-1j/2)*(Z@Rz(A1)@Ry(A0)@Rz(A2))
    return dA1
def DA2 (A0,A1,A2):
    dA2 = (-1j/2)*(Rz(A1)@Ry(A0)@Z@Rz(A2))
    return dA2

# defining inverse
def inverse(M):
    M_inverse = M.transpose().conjugate()
    return M_inverse



In [70]:
# function to define RHS for each unitary
def Utbm(j, A, U_goal): 
# j = unitary position, A = array of parameters, U_goal = U to be computed
    
    U1 = U(A[0,0],A[0,1],A[0,2])
    U2 = U(A[1,0],A[1,1],A[1,2])
    U3 = U(A[2,0],A[2,1],A[2,2])
    U4 = U(A[3,0],A[3,1],A[3,2])
    U5 = U(A[4,0],A[4,1],A[4,2])
    U6 = U(A[5,0],A[5,1],A[5,2])
    U7 = U(A[6,0],A[6,1],A[6,2])
    U8 = U(A[7,0],A[7,1],A[7,2])
    Ulist = [U1,U2,U3,U4,U5,U6,U7,U8]
    
    Ulist[j-1] = np.eye(2) # isolating unitary of interest
    
    U12 = np.kron(Ulist[0],Ulist[1])
    U34 = np.kron(Ulist[2],Ulist[3])
    U56 = np.kron(Ulist[4],Ulist[5])
    U78 = np.kron(Ulist[6],Ulist[7])

    # improvement: modulation possible
    if j==1:
        Uf = np.eye(4)
        Ub = U12@CNOT@U34@CNOT@U56@CNOT@U78
    elif j==2:
        Uf = U12
        Ub = CNOT@U34@CNOT@U56@CNOT@U78
    elif j==3:
        Uf = U12@CNOT
        Ub =U34@CNOT@U56@CNOT@U78
    elif j==4:
        Uf = U12@CNOT@U34
        Ub =CNOT@U56@CNOT@U78
    elif j==5:
        Uf = U12@CNOT@U34@CNOT
        Ub = U56@CNOT@U78
    elif j==6:
        Uf = U12@CNOT@U34@CNOT@U56
        Ub = CNOT@U78
    elif j==7:
        Uf = U12@CNOT@U34@CNOT@U56@CNOT
        Ub = U78
    elif j==8:
        Uf = U12@CNOT@U34@CNOT@U56@CNOT@U78
        Ub = np.eye(4)
    U_tbm = inverse(Uf)@U_goal@inverse(Ub) # U_tbm = rhs , U to be made
    return U_tbm

In [71]:
# function to do gradient descent
def gradient_descent(A, iterations,alpha,U_goal): # A = array of parameters, alpha - learning rate
    
    Aopt = np.copy(A) # to separate input and output
    stop_flag=0
    for j in range(1,9):
        # rhs of the optimization
        Ui = Utbm(j,Aopt,U_goal)

        # defining LHS 
        A0=Aopt[j-1][0]
        A1=Aopt[j-1][1]
        A2=Aopt[j-1][2]
        
        U_initial= U(A0,A1,A2)
        if j%2!=0:
            Uxyz = np.kron(U_initial,np.eye(2))
           
        else:
            Uxyz = np.kron(np.eye(2), U_initial)
            
        for i in range(iterations):
            #defining cost function
            Tr = np.trace(Ui@inverse(Uxyz))
            Abs = np.abs(Tr)
            Uerr = 4 - Abs
            print(Uerr)
            if Uerr < 1e-5:
                stop_flag=1
                break
            else:
                #gradient 
                dA0 = DA0(A0,A1,A2)
                dA1 = DA1(A0,A1,A2)
                dA2 = DA2(A0,A1,A2)
                if j%2!=0:
                    UdA0 = np.kron(dA0,np.eye(2))
                    UdA1 = np.kron(dA1,np.eye(2))
                    UdA2 = np.kron(dA2,np.eye(2))
                else:
                    UdA0 = np.kron(np.eye(2),dA0)
                    UdA1 = np.kron(np.eye(2),dA1)
                    UdA2 = np.kron(np.eye(2),dA2)

                Grad_A0 = -np.conj((Tr/Abs))*(np.trace(Ui@inverse(UdA0)))
                Grad_A0R = Grad_A0.real
                Grad_A1 = -np.conj((Tr/Abs))*(np.trace(Ui@inverse(UdA1)))
                Grad_A1R = Grad_A1.real
                Grad_A2 = -np.conj((Tr/Abs))*(np.trace(Ui@inverse(UdA2)))
                Grad_A2R = Grad_A2.real

                #updation
                A0 = A0 - alpha*Grad_A0R
                A1 = A1 - alpha*Grad_A1R
                A2 = A2 - alpha*Grad_A2R

                if j%2 != 0:
                    Uxyz = np.kron(U(A0,A1,A2),np.eye(2))
                else:
                    Uxyz = np.kron(np.eye(2), U(A0,A1,A2))
                Aopt[j-1][0]=A0
                Aopt[j-1][1]=A1
                Aopt[j-1][2]=A2
            if stop_flag==1:
                break

    return Aopt,stop_flag

In [ ]:
# random initialisations & function call

U_goal = unitary_group.rvs(4) # U to be achieved

A = np.zeros((8,3))
# randomly initialising the parameters
for i in range(8):
    for j in range(3):
        A[i][j] = random.uniform(0.0, 4*np.pi) 
AOpt =np.copy(A)

# truncation condition 
stop_flag=0
for i in range(1000):
    if stop_flag ==1:
        print("Truncation")
        break
    AOpt,stop_flag = gradient_descent(AOpt,10,0.01,U_goal)



2.3557179984645398
2.354877576325797
2.354043189078884
2.3532148017445564
2.3523923794796273
2.351575887576221
2.3507652914610695
2.3499605566948265
2.3491616489714113
2.3483685341173897
2.3475811780913642
2.3473526623215317
2.3471261926092772
2.3469017504393097
2.346679317443011
2.3464588753976225
2.3462404062254203
2.346023891992889
2.3458093149099035
2.3455966573288953
2.345385901744032
2.343962770285075
2.342551097617856
2.3411508051077883
2.3397618144254015
2.338384047549641
2.337017426771058
2.3356618746948765
2.334317314243952
2.332983668661619
2.331660861514429
2.330620249498426
2.3295864136781064
2.328559324296627
2.327538951398047
2.3265252648349724
2.3255182342760934
2.324517829213618
2.323524018970603
2.3225367727081796
2.32155605943268
2.319922410098704
2.318300792793372
2.3166911566702577
2.3150934503480514
2.313507621926189
2.3119336190003663
2.3103713886779373
2.308820877593183
2.3072820319224645
2.3057547973992363
2.3037721662063584
2.301792009182818
2.299814357187195


In [ ]:


    # #verification
    # U_12 = np.kron(U(AOpt[0][0], AOpt[0][1], AOpt[0][2]), U(AOpt[1][0], AOpt[1][1], AOpt[1][2]))
    # U_34 = np.kron(U(AOpt[2][0], AOpt[2][1], AOpt[2][2]), U(AOpt[3][0], AOpt[3][1], AOpt[3][2]))
    # U_56 = np.kron(U(AOpt[4][0], AOpt[4][1], AOpt[4][2]), U(AOpt[5][0], AOpt[5][1], AOpt[5][2]))
    # U_78 = np.kron(U(AOpt[6][0], AOpt[6][1], AOpt[6][2]), U(AOpt[7][0], AOpt[7][1], AOpt[7][2]))
     
    # U_found = U_12 @ CNOT @ U_34 @ CNOT @ U_56 @ CNOT @ U_78
    
    # print(U_goal@inverse(U_found))


1. How fast does it converge: Depends on the unitary - 10^4 iteration usually suffices
2. What's a stable learning rate - 0.01 seems to be okay - need to know how to verify
3. Does it reliably converge - Yes
4. Compare different unitaries and their convergence rate - how exactly ?